In [12]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd
import shutil

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')
 
# updating nipype from 1.7.0 -> 1.8.1, will hopefully ifx no PE estimates founds error # didn't fix

### Copes at level 3:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

In [13]:
## ALREADY DONE DONT RUN AGAIN
# # copy third level models so they all have same fwhm ('base') otherwise gets complicated with the inputs
# fwhms = ['1p5','1p6','1p5','3p125','3p0']
# datasets = ['Leipzig_7T_GdH', 'Leipzig_7T_SM', 'NTNU_7T_SJSI', 'aron_3T', 'openfmri_3T']

# for f, d in zip(fwhms,datasets):
#     these_data = sorted(glob.glob(f'/home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/{d}/model-0/model-0/model-0_fwhm-{f}_*'))
#     replaced_data = [x.replace(f,'base') for x in these_data]
#     for t, r in zip(these_data,replaced_data):
#         shutil.copy(t,r)

In [14]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

# datasets = ['Leipzig_7T_GdH','Leipzig_7T_SM','NTNU_7T_SJSI','aron_3T','openfmri_3T']

model_ns = ['0']
fwhms = ['base']

if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from third level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from third level model
    
datasets = [x.split('/')[-1]for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'group_level_model/*')))]

work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', 'fourth_level_model')
glm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'group_level_model')

template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "fourth_level_model", f'model-{model_ns[0]}')

templates = {'level3_cope': os.path.join(glm_folder, '*', 'model-{model_n}','model-{model_n}','model-{model_n}_fwhm-{fwhm}_subjectlevelcontrast-{contrast_n}_grouplevelcontrast-1_flame-12_desc-cope.nii.gz'),
             'level3_varcope': os.path.join(glm_folder, '*', 'model-{model_n}','model-{model_n}','model-{model_n}_fwhm-{fwhm}_subjectlevelcontrast-{contrast_n}_grouplevelcontrast-1_flame-12_desc-varcope.nii.gz'),
             'level3_tdof': os.path.join(glm_folder, '*', 'model-{model_n}','model-{model_n}','model-{model_n}_fwhm-{fwhm}_subjectlevelcontrast-{contrast_n}_grouplevelcontrast-1_flame-12_desc-tdof_t.nii.gz')}


In [15]:
print(f"""
datasets : {datasets}
fwhms : {fwhms[0]}
model_n : {model_ns}
no. datasets : {len(datasets)}
work_dir : {work_dir}
glm folder : {glm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level3_cope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level3_varcope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level3_tdof'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
""")


datasets : ['Leipzig_7T_GdH', 'Leipzig_7T_SM', 'NTNU_7T_SJSI', 'aron_3T', 'openfmri_3T']
fwhms : base
model_n : ['0']
no. datasets : 5
work_dir : /home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/fourth_level_model
glm folder : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model
output dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/fourth_level_model/model-0
contrasts : ['0', '1', '2', '3', '4', '5']

## if uneven number of template / designs / subs the code will crash
no.copes = 5
no.varvopes = 5
no.tdofs = 5



In [16]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [17]:
##
workflow = pe.Workflow(name='feat_level4_sst_hp')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level3_cope', copemerge, 'in_files')
workflow.connect(selector, 'level3_varcope', varcopemerge, 'in_files')

In [18]:
def get_dataset_ids():

    dataset_ids = ['Leipzig_7T_GdH', 'Leipzig_7T_SM', 'NTNU_7T_SJSI', 'aron_3T', 'openfmri_3T']
        
    return dataset_ids

dataset_id_getter = pe.Node(util.Function(output_names=['dataset_ids'],
                                          function=get_dataset_ids),
                            name='dataset_id_getter')

def get_design_matrix(third_level_contrast, dataset_ids): #=subject_ids):
    print(third_level_contrast)
    regressors = {'intercept': [1 for x in range(len(dataset_ids))]}
    
    # contrasts (3rd-level)
    fourth_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return fourth_level_contrasts, regressors


contrastgen_l4= pe.Node(util.Function(input_names=['third_level_contrast', 'dataset_ids'],
                                       output_names=['fourth_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l4')

level4model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l4model')


workflow.connect(dataset_id_getter, 'dataset_ids', contrastgen_l4, 'dataset_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l4, 'third_level_contrast')
workflow.connect(contrastgen_l4, 'fourth_level_contrasts', level4model, 'contrasts')
workflow.connect(contrastgen_l4, 'regressors', level4model, 'regressors')



In [19]:
# flameo 1 + 2
flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = True   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level4model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [20]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [21]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir


# substitutions_regexp = [(r'fourth_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_base_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
#                          'model-\\2/model-\\2_fwhm-base_datasetlevelcontrast-\\1_grouplevelcontrast-\\7_flame-\\3_desc-\\6_voxelthreshold-\\4.nii.gz'),
#                         (r'fourth_level_model/level4_.*/_contrast_n_(\d+)_fwhm_base_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
#                           'model-\\2/model-\\2_fwhm-base_datasetlevelcontrast-\\1_grouplevelcontrast-\\5_flame-\\3_desc-\\4.nii.gz'),
#                         # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
#                         #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
#                        ]

substitutions_regexp = [(r'fourth_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{4})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                         'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                        (r'fourth_level_model/level4_.*/_contrast_n_(\d+)_fwhm_(\S{4})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                          'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                        # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                        #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                       ]

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'fourth_level_model.level4_zstats')
workflow.connect(flameo, 'copes', datasink, 'fourth_level_model.level4_copes')
workflow.connect(flameo, 'var_copes', datasink, 'fourth_level_model.level4_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'fourth_level_model.level4_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'fourth_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'fourth_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'fourth_level_model.grf_cluster_indices')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 100})

221003-12:10:00,704 nipype.workflow INFO:
	 Workflow feat_level4_sst_hp settings: ['check', 'execution', 'logging', 'monitoring']
221003-12:10:00,783 nipype.workflow INFO:
	 Running in parallel.
221003-12:10:00,786 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 100.00/100.00, Free processors: 10/10.
221003-12:10:00,860 nipype.workflow INFO:
	 [Node] Setting-up "feat_level4_sst_hp.dataset_id_getter" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/fourth_level_model/feat_level4_sst_hp/dataset_id_getter".
221003-12:10:00,861 nipype.workflow INFO:
	 [Node] Setting-up "feat_level4_sst_hp.get_volume" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/fourth_level_model/feat_level4_sst_hp/get_volume".
221003-12:10:00,861 nipype.workflow INFO:
	 [Node] Setting-up "feat_level4_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/fourth_level_mo

221003-12:10:02,862 nipype.workflow INFO:
	 [Node] Executing "copemerge" <nipype.interfaces.fsl.utils.Merge>
32221003-12:10:02,863 nipype.workflow INFO:
	 [Node] Executing "contrastgen_l4" <nipype.interfaces.utility.wrappers.Function>

410


221003-12:10:02,864 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l4", elapsed time 0.002124s.
5221003-12:10:02,866 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l4", elapsed time 0.003074s.

221003-12:10:02,866 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l4", elapsed time 0.00252s.
221003-12:10:02,866 nipype.workflow INFO:
	 [Node] Executing "copemerge" <nipype.interfaces.fsl.utils.Merge>
221003-12:10:02,866 nipype.workflow INFO:
	 [Node] Executing "varcopemerge" <nipype.interfaces.fsl.utils.Merge>
221003-12:10:02,866 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l4", elapsed time 0.002688s.

221003-12:10:02,868 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l4", elapsed time 0.004416s.
221003-12:10:02,869 nipyp

221003-12:10:08,795 nipype.workflow INFO:
	 [Job 20] Completed (feat_level4_sst_hp.copemerge).
221003-12:10:08,796 nipype.workflow INFO:
	 [Job 21] Completed (feat_level4_sst_hp.varcopemerge).
221003-12:10:08,797 nipype.workflow INFO:
	 [Job 22] Completed (feat_level4_sst_hp.copemerge).
221003-12:10:08,798 nipype.workflow INFO:
	 [Job 23] Completed (feat_level4_sst_hp.varcopemerge).
221003-12:10:08,798 nipype.workflow INFO:
	 [Job 26] Completed (feat_level4_sst_hp.l4model).
221003-12:10:08,799 nipype.workflow INFO:
	 [Job 27] Completed (feat_level4_sst_hp.l4model).
221003-12:10:08,800 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 8 jobs ready. Free memory (GB): 99.60/100.00, Free processors: 8/10.
                     Currently running:
                       * feat_level4_sst_hp.varcopemerge
                       * feat_level4_sst_hp.copemerge
221003-12:10:08,852 nipype.workflow INFO:
	 [Node] Setting-up "feat_level4_sst_hp.l4model" in "/home/scotti/projects/3t_7t_sst_comp